In [1]:
import numpy as np
import pandas as pd
import os
import sklearn
from sklearn.decomposition import TruncatedSVD


In [2]:
columns = ['user_id', 'item_id', 'rating', 'timestamp']
frame = pd.read_csv(r'C:\Users\Acer\Downloads\ml-100k\ml-100k\u.data', sep='\t', names=columns)
frame.head()

user_id  item_id  rating  timestamp
0      196      242       3  881250949
1      186      302       3  891717742
2       22      377       1  878887116
3      244       51       2  880606923
4      166      346       1  886397596

In [3]:
columns = ['item_id', 'movie title', 'release date', 'video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
          'Animation', 'Childrens', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
          'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

movies = pd.read_csv(r'C:\Users\Acer\Downloads\ml-100k\ml-100k\u.item', sep='|', names=columns, encoding='latin-1')
movie_names = movies[['item_id', 'movie title']]
movie_names.head()

item_id        movie title
0        1   Toy Story (1995)
1        2   GoldenEye (1995)
2        3  Four Rooms (1995)
3        4  Get Shorty (1995)
4        5     Copycat (1995)

In [4]:
combined_movies_data = pd.merge(frame, movie_names, on='item_id')
combined_movies_data.head(5)

user_id  item_id  rating  timestamp   movie title
0      196      242       3  881250949  Kolya (1996)
1       63      242       3  875747190  Kolya (1996)
2      226      242       5  883888671  Kolya (1996)
3      154      242       3  879138235  Kolya (1996)
4      306      242       5  876503793  Kolya (1996)

In [5]:
combined_movies_data.shape

(100000, 5)

In [6]:
top_10 = combined_movies_data.groupby("item_id")["rating"].count().sort_values(ascending = False).head(5)
top_10_df = pd.DataFrame(top_10)
#top_10_df

In [7]:
top10_movie_ids = top_10_df.index.values
#top10_movie_ids

In [8]:
values = [value for value in top10_movie_ids]
#print(values)
print(type(values))

<class 'list'>


In [9]:
index = 0
top_movies =[]
for value in values:
    filter = combined_movies_data['item_id']==value
    top_movies = np.concatenate((top_movies,combined_movies_data[filter]['movie title'].unique()))

In [10]:
#top_movies 

In [11]:
#Build utility matrix

In [12]:
rating_crosstab = combined_movies_data.pivot_table(values = "rating" , index = "user_id",columns = "movie title",fill_value = 0)
#rating_crosstab.head(10)

In [13]:
#transposing utility matrix

In [14]:
rating_crosstab.shape

(943, 1664)

In [15]:
X = rating_crosstab.values.T
X.shape

(1664, 943)

In [16]:
#decomposing the utility matrix

In [17]:
SVD = TruncatedSVD(n_components=12, random_state=17)

In [18]:
SVD_matrix = SVD.fit_transform(X)
SVD_matrix.shape

(1664, 12)

In [19]:
#calculating correltion

In [20]:
corr_mat = np.corrcoef(SVD_matrix)
corr_mat.shape

(1664, 1664)

In [21]:
#isolating starwars

In [22]:
movie_names = rating_crosstab.columns
movies_list = list(movie_names)

In [23]:
star_wars = movies_list.index('Star Wars (1977)')
star_wars

1398

In [24]:
movie_index = []
type(movie_index)
for movie in top_movies:
    movie_index.append(movies_list.index(movie))

In [25]:
#movie_index

In [26]:
corr_star_wars = corr_mat[star_wars]
corr_star_wars.shape

(1664,)

In [32]:
related_movies =[]
for index in movie_index:
    corr_matrix = corr_mat[index]
    related_movies.append(list(movie_names[(corr_matrix<1.0) & (corr_matrix>=0.90)]))

In [33]:
recommendations = pd.DataFrame(related_movies)
recommendations.index = top_movies
recommendations.columns = ['recommendation'] * len(recommendations.columns)
recommendations

recommendation  \
Star Wars (1977)                      Die Hard (1988)   
Contact (1997)             Alien: Resurrection (1997)   
Fargo (1996)                             Bound (1996)   
Return of the Jedi (1983)             Die Hard (1988)   
Liar Liar (1997)                      Anaconda (1997)   

                                            recommendation  \
Star Wars (1977)           Empire Strikes Back, The (1980)   
Contact (1997)                              Contact (1997)   
Fargo (1996)                       Dead Man Walking (1995)   
Return of the Jedi (1983)  Empire Strikes Back, The (1980)   
Liar Liar (1997)                           Daylight (1996)   

                                    recommendation  \
Star Wars (1977)              Fugitive, The (1993)   
Contact (1997)                       Kissed (1996)   
Fargo (1996)                          Fargo (1996)   
Return of the Jedi (1983)     Fugitive, The (1993)   
Liar Liar (1997)           Devil's Own, The (1997)   

                                           recommendation  \
Star Wars (1977)           Raiders of the Lost Ark (1981)   
Contact (1997)                   Starship Troopers (1997)   
Fargo (1996)                        Godfather, The (1972)   
Return of the Jedi (1983)             Men in Black (1997)   
Liar Liar (1997)                     Jungle2Jungle (1997)   

                                           recommendation  \
Star Wars (1977)                Return of the Jedi (1983)   
Contact (1997)                 Tomorrow Never Dies (1997)   
Fargo (1996)                                  Heat (1995)   
Return of the Jedi (1983)  Raiders of the Lost Ark (1981)   
Liar Liar (1997)                         Liar Liar (1997)   

                                              recommendation  \
Star Wars (1977)           Terminator 2: Judgment Day (1991)   
Contact (1997)                                Volcano (1997)   
Fargo (1996)                        Leaving Las Vegas (1995)   
Return of the Jedi (1983)          Return of the Jedi (1983)   
Liar Liar (1997)                                Metro (1997)   

                                       recommendation  \
Star Wars (1977)               Terminator, The (1984)   
Contact (1997)                                   None   
Fargo (1996)                         Lone Star (1996)   
Return of the Jedi (1983)  Rumble in the Bronx (1995)   
Liar Liar (1997)                    Saint, The (1997)   

                                            recommendation  \
Star Wars (1977)                          Toy Story (1995)   
Contact (1997)                                        None   
Fargo (1996)                       Mighty Aphrodite (1995)   
Return of the Jedi (1983)  Star Trek: First Contact (1996)   
Liar Liar (1997)            Speed 2: Cruise Control (1997)   

                                               recommendation  \
Star Wars (1977)                                         None   
Contact (1997)                                           None   
Fargo (1996)               People vs. Larry Flynt, The (1996)   
Return of the Jedi (1983)                    Star Wars (1977)   
Liar Liar (1997)                               Volcano (1997)   

                                              recommendation  \
Star Wars (1977)                                        None   
Contact (1997)                                          None   
Fargo (1996)                                 Swingers (1996)   
Return of the Jedi (1983)  Terminator 2: Judgment Day (1991)   
Liar Liar (1997)                                        None   

                                   recommendation         recommendation  
Star Wars (1977)                             None                   None  
Contact (1997)                               None                   None  
Fargo (1996)                 Trainspotting (1996)  Twelve Monkeys (1995)  
Return of the Jedi (1983)  Terminator, The (1984)       Toy Story (1995)  
Liar Liar (1

In [34]:
#recommendations.to_csv('recommendations.csv', sep='\t',encoding = None ,line_terminator='\n')

In [30]:
#%pwd

In [31]:
# following movies should be recommended to a user who watches star wars 